# Tips n Tricks

## Training

In [ ]:
import torch
from tqdm.notebook import trange
import matplotlib.pyplot as plt
import seaborn as sns

sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)

%matplotlib inline

### Learning Rate

#### Cosine Annealing

In [ ]:
num_epochs = 300

model = torch.nn.Linear(2, 1)
optimizer = torch.optim.Adam(model.parameters(), lr=10)

# Cosine Annealing
schedulers = {}
schedulers["cosneal"] = [
    optimizer,
    torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0),
]

# Cosine Annealing with Warmups
model2 = torch.nn.Linear(2, 1)
optimizer2 = torch.optim.Adam(model.parameters(), lr=10)
schedulers["cosnealwarm"] = [
    optimizer2,
    LinearWarmupCosineAnnealingLR(
        optimizer2,
        warmup_epochs=10,
        max_epochs=num_epochs,
    ),
]

# One Cycle
model3 = torch.nn.Linear(2, 1)
optimizer3 = torch.optim.Adam(model.parameters(), lr=10)
schedulers["1cycle"] = [
    optimizer3,
    torch.optim.lr_scheduler.OneCycleLR(
        optimizer3, max_lr=10, epochs=num_epochs, steps_per_epoch=1
    ),
]


# Exponential
model4 = torch.nn.Linear(2, 1)
optimizer4 = torch.optim.Adam(model.parameters(), lr=10)
schedulers["exp"] = [
    optimizer4,
    torch.optim.lr_scheduler.ExponentialLR(
        optimizer4,
        gamma=0.95,
    ),
]

In [ ]:
lrs = {}

for imethod, (ioptimizer, ischeduler) in schedulers.items():
    lrs[imethod] = list()
    for i in trange(num_epochs):
        ioptimizer.step()
        lrs[imethod].append(ioptimizer.param_groups[0]["lr"])
        ischeduler.step()

In [ ]:
fig, ax = plt.subplots()

ax.plot(lrs["cosneal"], label="Cosine Annealing")
ax.plot(lrs["cosnealwarm"], label="Cosine Annealing + Lin Warmup")
ax.plot(lrs["1cycle"], label="OneCycle")
ax.plot(lrs["exp"], label="Exponential")
ax.set(xlabel="Epoch", ylabel="Learning Rate")
plt.legend(fontsize=12, framealpha=0.2)
plt.show()

#### Cosine Annealing + Linear Warmup

In [ ]:
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR

In [ ]:
scheduler = LinearWarmupCosineAnnealingLR(
    optimizer,
    warmup_epochs=25,
    max_epochs=num_epochs,
)

In [ ]:
lrs = []

for i in trange(num_epochs):
    optimizer.step()
    lrs.append(optimizer.param_groups[0]["lr"])
    scheduler.step()


fig, ax = plt.subplots()

ax.plot(lrs)
ax.set(xlabel="epochs", ylabel="learning rate")
plt.show()